In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df = pd.read_csv('Loan_Default.csv')
df = df.drop(columns=['ID', 'year'])


In [129]:
for i in df.columns:
    if(df[i].dtype not in ['object', 'str']):
        df[i].fillna(df[i].mean(), inplace=True)
        
df.dropna(inplace=True)

In [130]:
for i in df.columns:
    if(df[i].dtype in ['object', 'str'] and i != 'Status'):
        df = pd.concat([pd.get_dummies(df[i]).astype('int64'), df.drop(columns=i)], axis=1)


In [155]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

target = np.array(df['Status'])
features = StandardScaler().fit_transform(np.array(df.drop(columns='Status')))

xtrain, xtest, ytrain, ytest = train_test_split(features, target, test_size=0.2, random_state=42)


In [156]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

In [157]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(xtrain.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu',  kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu',  kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\Vugar\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [175]:
train = model.fit(
    xtrain, ytrain,
    epochs=10,
    batch_size=32,
    validation_data=(xtest, ytest)
)

Epoch 1/10
3600/3600 [==============================] - 18s 5ms/step - loss: 0.0243 - accuracy: 0.9960 - val_loss: 0.0176 - val_accuracy: 0.9997
Epoch 2/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.0248 - accuracy: 0.9962 - val_loss: 0.0139 - val_accuracy: 0.9992
Epoch 3/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.0228 - accuracy: 0.9962 - val_loss: 0.0128 - val_accuracy: 0.9992
Epoch 4/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.0241 - accuracy: 0.9963 - val_loss: 0.0135 - val_accuracy: 0.9996
Epoch 5/10
3600/3600 [==============================] - 14s 4ms/step - loss: 0.0217 - accuracy: 0.9966 - val_loss: 0.0103 - val_accuracy: 0.9994
Epoch 6/10
3600/3600 [==============================] - 14s 4ms/step - loss: 0.0227 - accuracy: 0.9966 - val_loss: 0.0131 - val_accuracy: 0.9991
Epoch 7/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.0201 - accuracy: 0.9970 - val_loss: 0.0117 - val_ac

In [177]:
model.evaluate(xtest, ytest)

900/900 [==============================] - 1s 1ms/step - loss: 0.0105 - accuracy: 0.9994


[0.010542315430939198, 0.9993749260902405]

In [178]:
from sklearn.metrics import confusion_matrix, f1_score

y_pred = model.predict(xtest)
y_pred = (y_pred > 0.5)

cm = confusion_matrix(ytest, y_pred)
print(cm)

f1_score(ytest, y_pred)

900/900 [==============================] - 1s 1ms/step
[[21698     9]
 [    9  7081]]


0.9987306064880113